In [ ]:
#import BruinKeijman as bk
import pandas as pd
import numpy as np
import scipy.constants as constants
from datetime import datetime, timedelta
import os 
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import io
from nbformat import current
from scipy.stats import kendalltau
import seaborn as sns
from scipy import stats
%matplotlib inline

In [ ]:
sns.set(style='ticks')#, rc={'axes.facecolor':'black', 'figure.facecolor':'black'})

In [ ]:
def inRaster(file_raster):
    raster = gdal.Open(file_raster, gdal.GA_ReadOnly)
    band = raster.GetRasterBand(1)
    array = band.ReadAsArray()
    extent = raster.GetGeoTransform()
    return raster, array, extent

In [ ]:
def saveRaster(path, array, dsSource, datatype=6, formatraster="GTiff", nan=None): 
    """
    Datatypes:
    unknown = 0
    byte = 1
    unsigned int16 = 2
    signed int16 = 3
    unsigned int32 = 4
    signed int32 = 5
    float32 = 6
    float64 = 7
    complex int16 = 8
    complex int32 = 9
    complex float32 = 10
    complex float64 = 11
    float32 = 6, 
    signed int = 3
    
    Formatraster:
    GeoTIFF = GTiff
    Erdas = HFA (output = .img)
    OGC web map service = WMS
    png = PNG
    """
    # Set Driver
    format_ = formatraster #save as format
    driver = gdal.GetDriverByName( format_ )
    driver.Register()
    
    # Set Metadata for Raster output
    cols = dsSource.RasterXSize
    rows = dsSource.RasterYSize
    bands = dsSource.RasterCount
    datatype = datatype#band.DataType
    
    # Set Projection for Raster
    outDataset = driver.Create(path, cols, rows, bands, datatype)
    geoTransform = dsSource.GetGeoTransform()
    outDataset.SetGeoTransform(geoTransform)
    proj = dsSource.GetProjection()
    outDataset.SetProjection(proj)
    
    # Write output to band 1 of new Raster and write NaN value
    outBand = outDataset.GetRasterBand(1)
    if nan != None:
        outBand.SetNoDataValue(nan)
    outBand.WriteArray(array) #save input array
    #outBand.WriteArray(dem)
    
    # Close and finalise newly created Raster
    #F_M01 = None
    outBand = None
    proj = None
    geoTransform = None
    outDataset = None
    driver = None
    datatype = None
    bands = None
    rows = None
    cols = None
    driver = None
    array = None

In [ ]:
def bk_lamda(T_a_avg):
    """
    input:
    T_a_avg     :  air temperature, gemiddelde dagtemperatuur (°C)
    
    output:
    _lambda :  the latent heat of vaporization, verdampingswarmte van water (MJ kg-1)
    """
    lamda = 2.501 - 2.3601e-3 * T_a_avg
    
    return lamda

In [ ]:
def bk_gamma(e_a, lamda):
    """
    input:
    e_a     :  vapor pressure at the air temperature, luchtdruk (kPa) op hoogte z0
    lamda   :  the latent heat of vaporization, verdampingswarmte van water (MJ kg-1)
    
    output:
    gamma   :  psychrometric constant, psychrometerconstante (kPa K-1)
    """
    
    gamma = 0.00163 * (e_a / lamda)
    
    return gamma

In [ ]:
def bk_s(e_s, T_a_avg):
    """
    input:
    e_s     :  saturated vapor pressure at the air temperature, verzadigde dampdruk (kPa)
    T_a_avg :  air temperature, gemiddelde dagtemperatuur (K)
    
    output:
    s       :  afgeleide van e_s bij luchttemperatuur T (kPa K-1)
    """    
    s = (4098 * e_s) / (T_a_avg + 237.3)**2
    return s

In [ ]:
def bk_Makkink(s, K_in, gamma, lamda_w):
    """
    input:
    s       :  afgeleide van e_s bij luchttemperatuur T (kPa K-1)
    k_in    :  inkomende kortgolvige straling (W/m2)
    gamma   :  psychrometric constant, psychrometerconstante (kPa K-1)
    lamda_w :  verdampingswarmte van water = 2.45 MJ kg-1
    
    output:
    E_mk    :  referentieverdamping (mm/d)
    """    
    lamda_w  = (lamda_w * 1000000 ) #J/Kg
    E_mk     = (((0.65 * (s/(s+gamma))) * K_in ) / lamda_w) * 86400 #mm/s -> mm/d
    
    return E_mk

In [ ]:
def bk_G(rho_w, c_w, T_vwt):
    """
    input:
    rho_w   :  density of air, dichtheid van water (= 1000 kg m-3)
    c_w     :  specific heat of water, soortelijke wartme van water (= 4200 J kg-1 K-1)
    T_vwt   :  vertical water temperature profile, change of water temperature over time, verandering van de watertemperatuur (K) met de tijd (s)
    
    output:
    G       :  soil heat flux density of land surfaces, bodemwarmtestroom (W m-2)
    """     

    G = rho_w * c_w * T_vwt
    return G

In [ ]:
# Tabel A1: schatting van de watertemperatuur en de verandering van watertemperatuur 
# met de tijd in friesland door het jaar heen
month = pd.Series(data=['january','february','march','april','may','june','july','august','september','october',
                        'november','december'],name='month')
DOY_s = pd.Series(data=[1,32,60,91,121,152,182,213,244,274,305,335],name='DOY_s')
DOY_e = pd.Series(data=[31,59,90,120,151,181,212,243,273,304,334,365],name='DOY_e')
T_w_C = pd.Series(data=[3,4,7,10,15,18,19,20,17,14,8,5],name='T_w_C')
T_w_K = pd.Series(data=[276.15,277.15,280.15,283.15,288.15,291.15,292.15,293.15,290.15,287.15,281.15,278.15],name='T_w_K')
dTdt_Kd = pd.Series(data=[-0.06,0.03,0.1,0.1,0.17,0.1,0.03,0.03,-0.1,-0.1,-0.19,-0.1],name='dTdt_Kd')
dTdt_Ks = pd.Series(data=[-7.46714e-7,3.73357e-7,1.19732e-6,1.12007e-6,1.92901e-6,1.12007e-6,3.85802e-7,3.73357e-7,
                          -1.12007e-6,-1.15741e-6,-2.24014e-6,-1.15741e-6],name='dTdt_Ks')

lookup_dTdt_Ks = pd.DataFrame([month,DOY_s,DOY_e,T_w_C,T_w_K,dTdt_Kd,dTdt_Ks]).T

def bk_T_vwt(z_avg, x, lookup_dTdt_Ks):
    """
    input:
    z_avg   :  the average depth of the lake area, waterdiepte (m) (Leeuwarden: gemiddelde waterdiepte over het gebied)
    x       :  the day of the the year (DOY), dag van het jaar
    lookup_dTdt_Ks  :  lookup table for estimate delta water temperature over delta time, verandering van de 
                       watertemperatuur (K) met de tijd (s) 
    
    output:
    T_vwt   :  vertical water temperature profile, change of water temperature over time, verandering van de watertemperatuur (K) met de tijd (s)
    """
    
    for idx, item in enumerate(lookup_dTdt_Ks['DOY_e']):        
        if x <= item:            
            _dTdt_Ks = lookup_dTdt_Ks.ix[idx]['dTdt_Ks']
            T_vwt = z_avg * _dTdt_Ks
            return T_vwt

In [ ]:
def bk_N_rel(C,n=None,N=None):
    """
    input:
    n       :  actual sunshine duration, actuele zonneschijnduur (uur d-1)
    N       :  maxium sunshine duration, maximaal mogelijke zonneschijnduur (uur d-1)
    C       :  bedekkingsgraad van de bewolking (-)
    
    output:
    N_rel   :  relatieve zonneschijnduur (-)
    """    
    
    #N_rel = n / N
    N_rel = 1 - C
    
    return N_rel

In [ ]:
def bk_e_s(T_a_min,T_a_max):
    """
    input:
    T_a_min :  minimum dagtemperatuur (°C)
    T_a_max :  maximum dagtemperatuur (°C)
    
    output:
    e_s    :  saturated vapor pressure at the air temperature, verzadigde dampdruk (kPa)
    """    
    
    e_s = 0.305 * ( np.exp((17.27 * T_a_min) / (T_a_min + 273.3)) + np.exp((17.27 * T_a_max) / (T_a_max + 273.3)) )
    return e_s

In [ ]:
def bk_e_z(RH, e_s):
    """
    input:
    RH      :  relatieve vochtigheid (-)
    e_s     :  verzadigde dampdruk (kPa)
    
    output:
    e_z     :  e(z) partiele dampdruk van water op hoogte z (kPa) (Leeuwarden: z = 1.5m)
    """
    
    e_z = RH * e_s
    return e_z

In [ ]:
def bk_L_netto_station(epsilon, sigma, T_a_max, T_a_min, e_z, N_rel):
    """
    input:
    epsilon :  emissivity, defauly emissivity of water = 0.98     
    sigma   :  Stefan-Boltzman constante = 4.903e-9 MK K-4 m-2 d-1
    T_a_min :  minimum dagtemperatuur (K)
    T_a_max :  maximum dagtemperatuur (K)    
    e_z     :  e(z) partiele dampdruk van water op hoogte z (kPa) (Leeuwarden: z = 1.5m)
    N_rel   :  relatieve zonneschijnduur (-)    
    
    output:
    L_netto_Wm_m2    :  netto langgolvige straling (W m-2) : 1 MJ m-2 d-1 = 11,574 W m-2
    L_netto_MJ_m2_d1 :  netto langgolvige straling (MJ m-2 d-1)    
    firstTerm        :  uitgaande langgolvige straling (MJ m-2 d-1) 
    secondTerm
    thirdTerm
    """    
    
    firstTerm        = epsilon * sigma * ((T_a_max**4 + T_a_min**4)/2) # L_uit_MJ_m2_d1
    secondTerm       = 0.34 - (0.14 * (e_z**0.5))
    thirdTerm        = 0.1 + (0.9 * N_rel)
    L_netto_MJ_m2_d1 = firstTerm * secondTerm * thirdTerm
    L_netto_Wm_m2    = L_netto_MJ_m2_d1 * 11.574
    
    return L_netto_Wm_m2, L_netto_MJ_m2_d1, firstTerm, secondTerm, thirdTerm

In [ ]:
def bk_L_netto_InLW(epsilon, sigma, T_a_max, T_a_min, L_inLW):
    """
    input:
    epsilon :  emissivity, defauly emissivity of water = 0.96 
    sigma   :  Stefan-Boltzman constante = 4.903e-9 MJ K-4 m-2 d-1
    T_a_min :  minimum dagtemperatuur (K)
    T_a_max :  maximum dagtemperatuur (K)    
    L_inLW  :  inkomende langgolvige straling (W m-2)
    
    output:
    L_netto_Wm_m2    :  netto langgolvige straling (W m-2) : 1 MJ m-2 d-1 = 11,574 W m-2
    L_uit_Wm_m2      :  uitgaande langgolvige straling (Wm_m2) 
    L_uit_MJ_m2_d1   :  uitgaande langgolvige straling (MJ m-2 d-1) 
    
    """    
    L_uit_MJ_m2_d1 = epsilon * sigma * ((T_a_max**4 + T_a_min**4)/2)
    L_uit_Wm_m2    = L_uit_MJ_m2_d1 * 11.574
    L_netto_Wm_m2  = L_uit_Wm_m2 - L_inLW    
    
    return L_netto_Wm_m2, L_uit_Wm_m2, L_uit_MJ_m2_d1

In [ ]:
def bk_Q_ask(K_in, alpha, L_netto):
    """
    input:
    K_in    :  dagsom globale straling (W m-2) :: inkomende kortgolvige straling (W m-2)
    alpha   :  albedo of water (voor water = 0.06)
    L_netto :  netto langgolvige straling (MJ m-2 d-1) : 1 MJ m-2 d-1 = 11574 W m-2
    
    output:
    Q_ask   :  Q* nettostraling (W m-2)    
    """   
    
    Q_ask = (1 - alpha) * K_in - L_netto
    return Q_ask

In [ ]:
def bk_lamdaE(Q_ask, G, s, gamma, alpha_aps, beta):
    """
    input:
    Q_ask   :  Q* nettostraling (W m-2)    
    G       :  soil heat flux density of land surfaces, bodemwarmtestroom (W m-2)
    s       :  afgeleide van e_s bij luchttemperatuur T (kPa K-1)
    gamma   :  psychrometric constant, psychrometerconstante (kPa K-1)
    alpha_aps : α' De Bruin-Keijman constante = 1.1
    beta    :  factor 10 W m-2
    
    output:
    lamdaE :  latent heat flux, latente warmtestroom (W m-2)
    """
    
    lamdaE = alpha_aps * (s / (s + gamma)) * (Q_ask - G) + beta
    return lamdaE

In [ ]:
def bk_E_ow(lamdaE, lamda_w = 2.45):
    """
    input:
    lamdaE :  latent heat flux, latente warmtestroom (W m-2)
    lamda_w:  verdampingswarmte van water = 2.45 MJ kg-1
    
    output:
    E_ow    :  openwaterverdamping (mm)
    """
    # 1 MJ m-2 d-1 = 11.574 W m-2
    lamda_w_Jkg = lamda_w * 1000000
    
    E_ow = (lamdaE / lamda_w_Jkg) * 86400  # multiply 1000 to go from meter to milimeter
    return E_ow

In [ ]:
# a latent heat flux of 100 Wm-2 should be equal to 3.5mm open water evaporation
#E_ow = bk._E_ow(100)
#print (E_ow)

In [ ]:
def r2(x, y):
    return stats.pearsonr(x, y)[0] ** 2

In [ ]:
def crop(image, x1, x2, y1, y2):
    """
    Return the cropped image at the x1, x2, y1, y2 coordinates
    """
    if x2 == -1:
        x2=image.shape[1]-1
    if y2 == -1:
        y2=image.shape[0]-1

    mask = np.zeros(image.shape)
    mask[y1:y2+1, x1:x2+1]=1
    m = mask>0

    return image[m].reshape((y2+1-y1, x2+1-x1))

In [ ]:
def plot_colors_labels(ax, cb, im, im_title, cb_label, fg_color='white', bg_color='black'):    
    # IMSHOW    
    # set title plus title color
    ax.set_title(im_title, color=fg_color)
    
    # set figure facecolor
    ax.patch.set_facecolor(bg_color)
    
    # set tick and ticklabel color
    im.axes.tick_params(color=fg_color, labelcolor=fg_color)
    
    # set imshow outline
    for spine in im.axes.spines.values():
        spine.set_edgecolor(fg_color)    

    # COLORBAR
    # set colorbar label plus label color
    cb.set_label(cb_label, color=fg_color)
    
    # set colorbar tick color
    cb.ax.yaxis.set_tick_params(color=fg_color)
    
    # set colorbar edgecolor 
    cb.outline.set_edgecolor(fg_color)
    
    # set colorbar ticklabels
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color=fg_color)

In [ ]:
def listall(RootFolder, varname='',extension='.png'):
    lists = [os.path.join(root, name)
             for root, dirs, files in os.walk(RootFolder)
             for name in files
             if varname in name
             if name.endswith(extension)]
    return lists

In [ ]:
def dictFromParam(rootDir):
    # input is the rootdirectory that contains the tif files
    # output is a dictionary with dates and absolute path to the items
    listDir = listall(rootDir, extension='.tif')

    # CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
    dates = []
    items = []
    for idx, item in enumerate(listDir):    
        base, file  = os.path.split(item)
        file, extension = os.path.splitext(file)
        date = datetime.strptime(file[-12:-4], '%Y%m%d')
        #print(date)
        dates.append(date)
        items.append(item)
    param_dict = dict(zip(dates, items))
    return param_dict

In [ ]:
baseDir = r'D:\jupyter\PR3492.10 Eact LSA-SAF\Toegepast op 2015'

In [ ]:
root = os.path.join(baseDir, 'ALBEDO', 'tif_IDW_100x100m')
listAlbedo = listall(root, extension='.tif')

# CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
dates_Albedo = []
items_Albedo = []
for idx, item in enumerate(listAlbedo):
    base, file  = os.path.split(listAlbedo[idx])
    file, extension = os.path.splitext(file)
    date = datetime.strptime(file[-12::], '%Y%m%d%H%M%S')
    #print(date)
    dates_Albedo.append(date)
    items_Albedo.append(item)
dict_Albedo = dict(zip(dates_Albedo, items_Albedo))

In [ ]:
root = os.path.join(baseDir, 'DIDSLF', 'tif_IDW_100x100m')
listInLW = listall(root, extension='.tif')

# CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
dates_InLW = []
items_InLW = []
for idx, item in enumerate(listInLW):
    base, file  = os.path.split(listInLW[idx])
    file, extension = os.path.splitext(file)
    date = datetime.strptime(file[-12::], '%Y%m%d%H%M%S')
    #print(date)
    dates_InLW.append(date)
    items_InLW.append(item)
dict_InLW = dict(zip(dates_InLW, items_InLW))

In [ ]:
root = os.path.join(baseDir, 'DIDSSF', 'tif_IDW_100x100m')
listInSW = listall(root, extension='.tif')

# CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
dates_InSW = []
items_InSW = []
for idx, item in enumerate(listInSW):
    base, file  = os.path.split(listInSW[idx])
    file, extension = os.path.splitext(file)
    date = datetime.strptime(file[-12::], '%Y%m%d%H%M%S')
    #print(date)
    dates_InSW.append(date)
    items_InSW.append(item)
dict_InSW = dict(zip(dates_InSW, items_InSW))

In [ ]:
# CREATE DICTIONARIES OF DATES AND ITEMS FOR KNMI FILES
# Tmean
rootTmean = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m','Tmean')
dict_Tmean = dictFromParam(rootTmean)

# Tmin
rootTmin = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Tmin')
dict_Tmin = dictFromParam(rootTmin)

# Tmax
rootTmax = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m','Tmax')
dict_Tmax = dictFromParam(rootTmax)

# Kmean
rootKmean = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Kmean')
dict_Kmean = dictFromParam(rootKmean)

# Kmax
rootKmax = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Kmax')
dict_Kmax = dictFromParam(rootKmax)

# Kmin
rootKmin = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Kmin')
dict_Kmin = dictFromParam(rootKmin)

# Nmean
rootNmean = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Nmean')
dict_Nmean = dictFromParam(rootNmean)

# Pmean
rootPmean = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Pmean')
dict_Pmean = dictFromParam(rootPmean)

# Qsum
rootQsum = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Qsum')
dict_Qsum = dictFromParam(rootQsum)

# Umean
rootUmean = os.path.join(baseDir, 'KNMI', 'tif_IDW_100x100m', 'Umean')
dict_Umean = dictFromParam(rootUmean)

In [ ]:
# load water mask or in fact load land and outer-sea mask
file_landgebruik = r'D:\jupyter\PR3492.10 Eact LSA-SAF\Toegepast op 2015\Landgebruik//Landgebruik.tif'
#file_water       = r'D:\Projects\Pr\3492.10\Water//Water_100x100m.tif'
array_water      = gdal.Open(file_landgebruik).ReadAsArray()

In [ ]:
fig = plt.figure(figsize=(14,4))
plt.subplot(131)
water = np.ma.masked_not_equal(array_water, 12000)
np.ma.set_fill_value(water, -999)
water[water == 12000] = 1
im = plt.imshow(water)
plt.colorbar(im)
plt.title('water == 1')
#plt.show()

plt.subplot(132)
land = np.ma.masked_where((array_water == 12000) | (array_water == 65535),array_water)
np.ma.set_fill_value(land, -999)
land[~land.mask] = 1
im = plt.imshow(land)
plt.colorbar(im)
plt.title('land == 1')

# grid met diepte van water, default is nu -3mNAP
plt.subplot(133)
z_avg = 3
array_z_avg = np.ma.masked_not_equal(array_water, 12000)
np.ma.set_fill_value(array_z_avg, -999)
array_z_avg[array_z_avg == 12000] = z_avg
im = plt.imshow(array_z_avg)
plt.colorbar(im)
plt.title('waterdiepte == 3')
plt.tight_layout()
plt.show()

In [ ]:
font = {'family': 'arial',
        'color':  'white',
        'weight': 'normal',
        'size': 16,
        }

In [ ]:
constanten = {}
constanten['albedo van water']             = {'waarde ':0.06, 'symbool':'α', 'eenheid':'-'}
constanten['Stefan-Bolzmann constante']    = {'waarde ':(constants.Stefan_Boltzmann * 86400) / 1000000, 'symbool':'σ', 'eenheid':'MJ/K4/m²/d'}
#constanten['Stefan-Bolzmann constante_W']    = {'waarde ':constants.Stefan_Boltzmann, 'symbool':'σ', 'eenheid':'W/K4/m²'}
constanten['dichtheid van water']          = {'waarde ':1000, 'symbool':'ρw', 'eenheid':'kg/m3'}
constanten['soortelijke warmte van water'] = {'waarde ':4200, 'symbool':'ϲw', 'eenheid':'J/kg/K'} 
constanten['Von Kaman constante']          = {'waarde ':0.41, 'symbool':'K', 'eenheid':'-'}
constanten['emissiviteit van water']       = {'waarde ':0.96, 'symbool':'ε', 'eenheid':'-'}
constanten['emissiviteit van land']        = {'waarde ':0.98, 'symbool':'ε', 'eenheid':'-'}
constanten['verdampingswarmte van water']  = {'waarde ':2.45, 'symbool':'λ', 'eenheid':'MJ/kg'}
constanten['hoogte windsnelheidsmetingen'] = {'waarde ':10, 'symbool':'zm', 'eenheid':'m'}
constanten['hoogte temperatuur- en vochtigheidsmetingen'] = {'waarde ':1.5, 'symbool':'zh', 'eenheid':'m'}
constanten['Priestley-Taylorconstante']    = {'waarde ':1.26, 'symbool':'α', 'eenheid':'-'}
constanten['De Bruin-Keijmanconstante']    = {'waarde ':1.1, 'symbool':"α'", 'eenheid':'-'}
constanten['factor']                       = {'waarde ':10, 'symbool':'β', 'eenheid':'W/m2'}

In [ ]:
variabelen = {}
variabelen['albedo']                                     = {'symbool':'a', 'eenheid':'-'}
variabelen['inkomende langgolvige straling']             = {'symbool':'L_in', 'eenheid':'W/m²'}
variabelen['uitgaande langgolvige straling']             = {'symbool':'L_uit', 'eenheid':'W/m²'}
variabelen['uitgaande langgolvige straling MJm2d1']      = {'symbool':'L_uit', 'eenheid':'MJ/m²/d'}
variabelen['netto langgolvige straling']                 = {'symbool':'L_netto', 'eenheid':'W/m²'}
variabelen['netto langgolvige straling station']         = {'symbool':'L_netto', 'eenheid':'W/m²'}
variabelen['netto langgolvige straling station MJm2d1']  = {'symbool':'L_netto', 'eenheid':'MJ/m²/d'}
variabelen['netto langgolvige straling station term1']   = {'symbool':'L_netto_term1', 'eenheid':'MJ/m²/d'}
variabelen['netto langgolvige straling station term2']   = {'symbool':'L_netto_term2', 'eenheid':'-'}
variabelen['netto langgolvige straling station term3']   = {'symbool':'L_netto_term3', 'eenheid':'-'}
variabelen['inkomende kortgolvige straling']             = {'symbool':'K_in', 'eenheid':'W/m²'}
variabelen['dagsom globale straling']                    = {'symbool':'K_in', 'eenheid':'W/m²'} 
variabelen['nettostraling']                              = {'symbool':'Q*', 'eenheid':'W/m²'}
variabelen['nettostraling station']                      = {'symbool':'Q*', 'eenheid':'W/m²'}
variabelen['gemiddelde dagtemperatuur in graden']        = {'symbool':'T', 'eenheid':'°C'}
variabelen['maximum dagtemperatuur in graden']           = {'symbool':'T_max', 'eenheid':'°C'}
variabelen['minimum dagtemperatuur in graden']           = {'symbool':'T_min', 'eenheid':'°C'}
variabelen['gemiddelde dagtemperatuur in Kelvin']        = {'symbool':'T', 'eenheid':'K'}
variabelen['maximum dagtemperatuur in Kelvin']           = {'symbool':'T_max', 'eenheid':'K'}
variabelen['minimum dagtemperatuur in Kelvin']           = {'symbool':'T_min', 'eenheid':'K'}
variabelen['verticaal water temperatuur profiel']        = {'symbool':'dT/dt', 'eenheid':'K/s'}
variabelen['bodemwarmtestroom']                          = {'symbool':'G', 'eenheid':'W/m²'} 
variabelen['psychrometerconstante']                      = {'symbool':'γ', 'eenheid':'kPa/K'}
variabelen['verzadigde dampdruk']                        = {'symbool':'e_sat', 'eenheid':'kPa'}
variabelen['partiele dampdruk']                          = {'symbool':'e_z', 'eenheid':'kPa'}
variabelen['relatieve vochtigheid']                      = {'symbool':'RH', 'eenheid':'-'}
variabelen['bedekkingsgraad bewolking']                  = {'symbool':'C', 'eenheid':'-'}
variabelen['luchtdruk']                                  = {'symbool':'p', 'eenheid':'kPa'}
variabelen['referentie gewasverdamping']                 = {'symbool':'E_makkink', 'eenheid':'mm/d'}
variabelen['latente warmtestroom']                       = {'symbool':'λE', 'eenheid':'W/m²'}
variabelen['openwaterverdamping']                        = {'symbool':'E', 'eenheid':'mm/d'}
variabelen['verdampingswarmte van water']                = {'symbool':'λ', 'eenheid':'MJ/kg'}
variabelen['afgeleide van e_sat bij luchttemperatuur T'] = {'symbool':'s', 'eenheid':'kPa/K'}


index   = dates_Albedo
columns = [x for x in variabelen]
df      = pd.DataFrame(index=index, columns=columns)
df      = df.fillna(0.) # with 0s rather than NaNs

In [ ]:
print ('Functies de Bruin-Keijmanmethode geladen, beschikbaar als bk_xxx')